## PREPROCESSING DATASET UNTUK SVM

In [1]:
from pyspark.sql import SparkSession
import spacy, pandas as pd
import os
import glob
from tqdm import tqdm

In [1]:
import pandas as pd
import nltk
import spacy
from google.colab import drive
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/openjdk11"
spark = SparkSession.builder.appName("Load Lemma Chunks").getOrCreate()

25/10/15 09:43:31 WARN Utils: Your hostname, mothership resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface enp3s0)
25/10/15 09:43:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/10/15 09:43:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

code cells ini mengunduh berbagai dataset dan lexical resources dari NLTK yang dibutuhkan untuk preprocessing teks sebelum training SVM, resource tersebut meliputi punkt untuk tokenisasi kalimat dan kata, averaged_perceptron_tagger_eng untuk part-of-speech tagging, serta wordnet dan omw-1.4 yang digunakan dalam proses lemmatization.

In [2]:
df = pd.read_parquet('./dataset/data_restaurant.parquet')

In [3]:
df1 = pd.read_csv('./lemma_chunk_2250000_2300000.csv')

In [3]:
df.count()

clean_text    4724684
sentiment     4724684
dtype: int64

**Data Post Transformation**

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/svm_data_pa/part-00000-27e2daf8-89c1-4f0a-93b6-51d02ed43deb-c000.snappy.parquet')

In [ ]:
df.count()

,0
clean_text,60000
sentiment,60000


In [6]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

lemmatizer = WordNetLemmatizer()

def lemmatize_with_pos(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged]
    return " ".join(lemmatized)

In [ ]:
df['lemma_text'] = df['clean_text'].apply(lemmatize_with_pos)

In [ ]:
df.head()

,clean_text,sentiment,lemma_text
0,if you decide to eat here just be aware it is ...,1,if you decide to eat here just be aware it be ...
1,a couple friends and i stopped by for some lat...,1,a couple friend and i stop by for some late ni...
2,sometimes this food is very very good unfortu...,1,sometimes this food be very very good unfortun...
3,after trying a few ramen places with crazy var...,1,after try a few ramen place with crazy variety...
4,great food terrible customer service ive been ...,1,great food terrible customer service ive be th...


code cells ini melakukan lemmatization pada teks, yaitu mengubah kata ke bentuk dasarnya dengan mempertimbangkan jenis katanya.

fungsi get_wordnet_pos menentukan tipe kata berdasarkan tag POS (seperti adjective, verb, noun, adverb) agar hasil lemmatization lebih akurat,

fungsi lemmatize_with_pos kemudian melakukan tokenisasi, memberi tag POS pada tiap kata, lalu menerapkan lemmatizer sesuai jenis katanya. Hasil akhirnya disimpan ke kolom baru lemma_text dalam DataFrame.

In [ ]:
def get_pos_tags(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    return tagged

In [ ]:
df['pos_tags'] = df['clean_text'].apply(get_pos_tags)

code cells ini menambahkan kolom baru berisi Part-of-Speech tags untuk setiap teks, fungsi get_pos_tags melakukan tokenisasi menggunakan word_tokenize, lalu memberi label jenis kata (seperti noun, verb, adjective, dll.) dengan pos_tag

In [ ]:
df.head()

,clean_text,sentiment,lemma_text,pos_tags
0,if you decide to eat here just be aware it is ...,1,if you decide to eat here just be aware it be ...,"[(if, IN), (you, PRP), (decide, VBP), (to, TO)..."
1,a couple friends and i stopped by for some lat...,1,a couple friend and i stop by for some late ni...,"[(a, DT), (couple, JJ), (friends, NNS), (and, ..."
2,sometimes this food is very very good unfortu...,1,sometimes this food be very very good unfortun...,"[(sometimes, RB), (this, DT), (food, NN), (is,..."
3,after trying a few ramen places with crazy var...,1,after try a few ramen place with crazy variety...,"[(after, IN), (trying, VBG), (a, DT), (few, JJ..."
4,great food terrible customer service ive been ...,1,great food terrible customer service ive be th...,"[(great, JJ), (food, NN), (terrible, JJ), (cus..."


In [ ]:
df[['clean_text','lemma_text','pos_tags','sentiment']].head(5)

,clean_text,lemma_text,pos_tags,sentiment
0,if you decide to eat here just be aware it is ...,if you decide to eat here just be aware it be ...,"[(if, IN), (you, PRP), (decide, VBP), (to, TO)...",1
1,a couple friends and i stopped by for some lat...,a couple friend and i stop by for some late ni...,"[(a, DT), (couple, JJ), (friends, NNS), (and, ...",1
2,sometimes this food is very very good unfortu...,sometimes this food be very very good unfortun...,"[(sometimes, RB), (this, DT), (food, NN), (is,...",1
3,after trying a few ramen places with crazy var...,after try a few ramen place with crazy variety...,"[(after, IN), (trying, VBG), (a, DT), (few, JJ...",1
4,great food terrible customer service ive been ...,great food terrible customer service ive be th...,"[(great, JJ), (food, NN), (terrible, JJ), (cus...",1


In [ ]:
df[['clean_text','lemma_text','pos_tags','sentiment']].to_parquet('/content/drive/MyDrive/svm_data_pa_preprocessing/svm_data_preprocessed.parquet')

**Data lemmetization full**

In [10]:
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
nlp.max_length = 3_000_000

texts = df["clean_text"].tolist()
docs = nlp.pipe(texts, n_process=12, batch_size=1000)

/home/parasite/.pyenv/versions/3.10.13/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
BATCH_SIZE = 50_000

for start in range(0, len(df), BATCH_SIZE):
    end = start + BATCH_SIZE
    chunk = df.iloc[start:end]
    
    chunk["lemma_text"] = [
        " ".join([token.lemma_ for token in doc]) 
        for doc in tqdm(nlp.pipe(chunk["clean_text"], n_process=12))
    ]
    
    chunk.to_csv(f"lemma_chunk_{start}_{end}.csv", index=False)
    print(f"Saved {start}-{end}")

50000it [01:59, 420.13it/s] 
/tmp/ipykernel_10631/4243060105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["lemma_text"] = [


Saved 0-50000


50000it [01:57, 426.75it/s] 
/tmp/ipykernel_10631/4243060105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["lemma_text"] = [


Saved 50000-100000


50000it [01:50, 450.67it/s] 
/tmp/ipykernel_10631/4243060105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["lemma_text"] = [


...

50000it [01:58, 420.28it/s] 
/tmp/ipykernel_10631/4243060105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["lemma_text"] = [


Saved 2200000-2250000


50000it [02:34, 322.82it/s] 
/tmp/ipykernel_10631/4243060105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["lemma_text"] = [


Saved 2250000-2300000


0it [00:00, ?it/s]

In [11]:
BATCH_SIZE = 50_000

done_chunks = {
    int(f.split("_")[2])
    for f in os.listdir()
    if f.startswith("lemma_chunk_") and f.endswith(".csv")
}

for start in range(0, len(df), BATCH_SIZE):
    if start in done_chunks:
        print(f"Skipping chunk {start} (already processed)")
        continue

    end = min(start + BATCH_SIZE, len(df))
    print(f"Processing chunk {start}-{end}")

    chunk = df.iloc[start:end].copy()

    chunk["lemma_text"] = [
        " ".join([token.lemma_ for token in doc])
        for doc in tqdm(nlp.pipe(chunk["clean_text"], n_process=12, batch_size=100))
    ]

    chunk.to_csv(f"lemma_chunk_{start}_{end}.csv", index=False)
    print(f"Saved {start}-{end}")

Skipping chunk 0 (already processed)
Skipping chunk 50000 (already processed)
Skipping chunk 100000 (already processed)
Skipping chunk 150000 (already processed)
Skipping chunk 200000 (already processed)
Skipping chunk 250000 (already processed)
Skipping chunk 300000 (already processed)
Skipping chunk 350000 (already processed)
Skipping chunk 400000 (already processed)
Skipping chunk 450000 (already processed)
Skipping chunk 500000 (already processed)
Skipping chunk 550000 (already processed)
Skipping chunk 600000 (already processed)
Skipping chunk 650000 (already processed)
Skipping chunk 700000 (already processed)
Skipping chunk 750000 (already processed)
Skipping chunk 800000 (already processed)
Skipping chunk 850000 (already processed)
Skipping chunk 900000 (already processed)
Skipping chunk 950000 (already processed)
Skipping chunk 1000000 (already processed)
Skipping chunk 1050000 (already processed)
Skipping chunk 1100000 (already processed)
Skipping chunk 1150000 (already proce

0it [00:00, ?it/s]

50000it [02:04, 402.71it/s] 


Saved 2300000-2350000
Processing chunk 2350000-2400000


50000it [01:48, 462.91it/s] 


...

Saved 4600000-4650000
Processing chunk 4650000-4700000


50000it [01:59, 417.53it/s] 


Saved 4650000-4700000
Processing chunk 4700000-4724684


24684it [01:14, 330.34it/s] 


Saved 4700000-4724684


In [3]:
input_path = "./" 
df = spark.read.csv(
    input_path + "lemma_chunk_*.csv", 
    header=True, 
    inferSchema=True
)

In [5]:
df.printSchema()
df.show(5)
df.count()

root
 |-- clean_text: string (nullable = true)
 |-- sentiment: integer (nullable = true)
 |-- lemma_text: string (nullable = true)

+--------------------+---------+--------------------+
|          clean_text|sentiment|          lemma_text|
+--------------------+---------+--------------------+
|i have grown up g...|        0|I have grow up go...|
|when we walked in...|        0|when we walk in w...|
|i came to paladar...|        2|I come to paladar...|
|excellent neighbo...|        2|excellent neighbo...|
|amazing definitel...|        2|amazing definitel...|
+--------------------+---------+--------------------+
only showing top 5 rows



14892622

In [6]:
df.coalesce(1).write.parquet("svm_data_lemma",mode="overwrite")